In [1]:
# Install necessary libraries
!pip install -q torch torchvision albumentations xmltodict matplotlib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.6 MB/s eta 0:00:00


In [2]:
import os
import glob
import xmltodict
import torch
import torch.nn as nn
import torchvision.models as models
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2


/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.7' (you have '2.0.5'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [3]:
# Create folder and download VOC 2007
!mkdir -p data
%cd data
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar
!tar -xf VOCtrainval_06-Nov-2007.tar
%cd ..


/kaggle/working/data
--2025-05-26 07:31:58--  http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar
Resolving host.robots.ox.ac.uk (host.robots.ox.ac.uk)... 129.67.94.152
Connecting to host.robots.ox.ac.uk (host.robots.ox.ac.uk)|129.67.94.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 460032000 (439M) [application/x-tar]
Saving to: ‘VOCtrainval_06-Nov-2007.tar’

VOCtrainval_06-Nov- 100%[===================>] 438.72M   141MB/s    in 3.1s    

2025-05-26 07:32:01 (141 MB/s) - ‘VOCtrainval_06-Nov-2007.tar’ saved [460032000/460032000]

/kaggle/working


In [4]:
class VOCDataset(Dataset):
    def __init__(self, root_dir, transforms=None):
        self.root_dir = root_dir
        self.transforms = transforms
        self.image_paths = sorted(glob.glob(os.path.join(root_dir, "JPEGImages", "*.jpg")))
        self.annotation_paths = sorted(glob.glob(os.path.join(root_dir, "Annotations", "*.xml")))

        self.class_names = [
            "aeroplane", "bicycle", "bird", "boat", "bottle",
            "bus", "car", "cat", "chair", "cow",
            "diningtable", "dog", "horse", "motorbike", "person",
            "pottedplant", "sheep", "sofa", "train", "tvmonitor"
        ]
        self.class_dict = {k: v for v, k in enumerate(self.class_names)}

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        anno_path = self.annotation_paths[idx]

        img = Image.open(img_path).convert("RGB")
        img = np.array(img)

        with open(anno_path) as f:
            anno = xmltodict.parse(f.read())["annotation"]

        boxes = []
        labels = []

        objects = anno.get("object", [])
        if not isinstance(objects, list):
            objects = [objects]

        for obj in objects:
            label = self.class_dict[obj["name"]]
            bbox = obj["bndbox"]
            box = [
                float(bbox["xmin"]),
                float(bbox["ymin"]),
                float(bbox["xmax"]),
                float(bbox["ymax"])
            ]
            boxes.append(box)
            labels.append(label)

        if self.transforms:
            transformed = self.transforms(image=img, bboxes=boxes, class_labels=labels)
            img = transformed["image"]
            boxes = transformed["bboxes"]
            labels = transformed["class_labels"]

        target = {"boxes": torch.tensor(boxes, dtype=torch.float32),
                  "labels": torch.tensor(labels, dtype=torch.int64)}

        return img, target


In [5]:
transform = A.Compose([
    A.Resize(448, 448),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels']))

dataset = VOCDataset("data/VOCdevkit/VOC2007", transforms=transform)
loader = DataLoader(dataset, batch_size=4, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))


In [6]:
class YOLOResNet(nn.Module):
    def __init__(self, num_classes=20, S=7, B=2):
        super(YOLOResNet, self).__init__()
        self.S = S
        self.B = B
        self.C = num_classes

        resnet = models.resnet34(pretrained=True)
        self.backbone = nn.Sequential(
    *list(resnet.children())[:-2],
    nn.AdaptiveAvgPool2d((S, S))  # This forces output to [B, 512, 7, 7]
)
        self.head = nn.Sequential(
            nn.Conv2d(512, 1024, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(1024, self.C + self.B * 5, kernel_size=1)  # Output: [B, 30, S, S]
        )


    def forward(self, x):
        x = self.backbone(x)
        x = self.head(x)
        return x.permute(0,2,3,1)

In [7]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
print(device)

cuda


In [9]:
class YOLOLoss(nn.Module):
    def __init__(self, S=7, B=2, C=20, lambda_coord=5, lambda_noobj=0.5):
        super(YOLOLoss, self).__init__()
        self.mse = nn.MSELoss(reduction="sum")
        self.S = S
        self.B = B
        self.C = C
        self.lambda_coord = lambda_coord
        self.lambda_noobj = lambda_noobj

    def forward(self, predictions, targets):
        # predictions: [B, S, S, 30]
        # targets: [B, S, S, 30]
        obj_mask = targets[..., 4] > 0  # Object present
        noobj_mask = targets[..., 4] == 0  # No object

        # === 1. Coordinate loss ===
        coord_loss = self.mse(
            predictions[obj_mask][..., 0:2],
            targets[obj_mask][..., 0:2]
        ) + self.mse(
            torch.sqrt(torch.abs(predictions[obj_mask][..., 2:4] + 1e-6)),
            torch.sqrt(torch.abs(targets[obj_mask][..., 2:4] + 1e-6))
        )

        # === 2. Objectness (confidence) loss ===
        obj_loss = self.mse(
            predictions[obj_mask][..., 4],
            targets[obj_mask][..., 4]
        )

        # === 3. No-object loss ===
        noobj_loss = self.mse(
            predictions[noobj_mask][..., 4],
            targets[noobj_mask][..., 4]
        )

        # === 4. Class loss ===
        class_loss = self.mse(
            predictions[obj_mask][..., 10:],
            targets[obj_mask][..., 10:]
        )

        total_loss = (
            self.lambda_coord * coord_loss +
            obj_loss +
            self.lambda_noobj * noobj_loss +
            class_loss
        )

        return total_loss


In [10]:
model = YOLOResNet(S=7)
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
#criterion = nn.MSELoss()  # Placeholder — use YOLO loss later
criterion=YOLOLoss(S=7, B=2, C=20)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 209MB/s]


#try with the yolo


In [11]:
def encode_yolo_targets(targets, S=7, B=2, C=20, img_size=448):
    batch_size = len(targets)
    target_tensor = torch.zeros((batch_size, S, S, B*5 + C))

    for b in range(batch_size):
        boxes = targets[b]['boxes']
        labels = targets[b]['labels']

        for i in range(len(boxes)):
            xmin, ymin, xmax, ymax = boxes[i]
            class_id = labels[i].item()

            # Convert to center x,y and width, height
            x_center = ((xmin + xmax) / 2) / img_size
            y_center = ((ymin + ymax) / 2) / img_size
            w = (xmax - xmin) / img_size
            h = (ymax - ymin) / img_size

            grid_x = int(x_center * S)
            grid_y = int(y_center * S)
            x_cell = x_center * S - grid_x
            y_cell = y_center * S - grid_y

            if target_tensor[b, grid_y, grid_x, 4] == 0:
                target_tensor[b, grid_y, grid_x, 0:5] = torch.tensor([x_cell, y_cell, w, h, 1])
                target_tensor[b, grid_y, grid_x, 10 + class_id] = 1
            elif target_tensor[b, grid_y, grid_x, 9] == 0:
                target_tensor[b, grid_y, grid_x, 5:10] = torch.tensor([x_cell, y_cell, w, h, 1])
                target_tensor[b, grid_y, grid_x, 10 + class_id] = 1

    return target_tensor


In [12]:
from tqdm import tqdm

def train_one_epoch(model, loader, optimizer, criterion, device, epoch, total_epochs):
    model.train()
    running_loss = 0.0
    loop = tqdm(enumerate(loader), total=len(loader), desc=f"Epoch [{epoch+1}/{total_epochs}]", leave=False)

    for batch_idx, (imgs, targets) in loop:
        imgs = torch.stack(imgs).to(device)
        targets_encoded = encode_yolo_targets(targets, S=7, B=2, C=20).to(device)

        preds = model(imgs)
        loss = criterion(preds, targets_encoded)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        avg_loss = running_loss / (batch_idx + 1)
        loop.set_postfix(loss=loss.item(), avg_loss=avg_loss)


In [13]:
from tqdm import tqdm

def train_one_epoch(model, loader, optimizer, criterion, device, epoch, total_epochs):
    model.train()
    running_loss = 0.0
    loop = tqdm(enumerate(loader), total=len(loader), desc=f"Epoch [{epoch+1}/{total_epochs}]", leave=False)

    for batch_idx, (imgs, targets) in loop:
        imgs = torch.stack(imgs).to(device)
        targets_encoded = encode_yolo_targets(targets, S=7, B=2, C=20).to(device)

        preds = model(imgs)
        loss = criterion(preds, targets_encoded)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        avg_loss = running_loss / (batch_idx + 1)
        loop.set_postfix(loss=loss.item(), avg_loss=avg_loss)


In [14]:
#above code in the function 
from torchmetrics.detection.mean_ap import MeanAveragePrecision
from torchvision.ops import box_iou

def evaluate_map_score(model, loader, device, conf_threshold=0.2, iou_threshold=0.5):
    model.eval()
    map_metric = MeanAveragePrecision(iou_type="bbox", iou_thresholds=[iou_threshold])

    with torch.no_grad():
        for imgs, targets in loader:
            imgs = torch.stack(imgs).to(device)
            outputs = model(imgs).cpu()

            for i in range(len(outputs)):
                pred = outputs[i]
                boxes = []
                scores = []
                labels = []

                for row in range(7):
                    for col in range(7):
                        cell = pred[row, col]
                        class_probs = cell[10:]
                        class_id = torch.argmax(class_probs).item()
                        class_score = class_probs[class_id].item()

                        for b in range(2):
                            x, y, w, h, conf = cell[b*5:b*5+5]
                            total_conf = conf.item() * class_score
                            if total_conf > conf_threshold:
                                cx = (col + x.item()) / 7
                                cy = (row + y.item()) / 7
                                bw = w.item()
                                bh = h.item()
                                x1 = cx - bw / 2
                                y1 = cy - bh / 2
                                x2 = cx + bw / 2
                                y2 = cy + bh / 2
                                boxes.append([x1 * 448, y1 * 448, x2 * 448, y2 * 448])
                                scores.append(total_conf)
                                labels.append(class_id)

                preds = [{
                    "boxes": torch.tensor(boxes),
                    "scores": torch.tensor(scores),
                    "labels": torch.tensor(labels)
                }]

                gts = [{
                    "boxes": targets[i]["boxes"],
                    "labels": targets[i]["labels"]
                }]

                map_metric.update(preds, gts)

    results = map_metric.compute()
    map_score = results["map_50"].item()
    return map_score


In [15]:
best_map = 0
patience = 10
epochs_since_improvement = 0

total_epochs = 100

for epoch in range(total_epochs):
    print(f"\n📘 Epoch {epoch+1}/{total_epochs}")

    # Train
    train_one_epoch(model, loader, optimizer, criterion, device, epoch, total_epochs)

    # Evaluate
    map_score = evaluate_map_score(model, loader, device, conf_threshold=0.2, iou_threshold=0.5)

    # Logging
    print(f"🔍 mAP@0.5 this epoch   : {map_score:.4f}")
    print(f"📈 Best mAP so far     : {best_map:.4f}")
    print(f"📌 Epochs w/o improve  : {epochs_since_improvement}/{patience}")

    if map_score > best_map:
        best_map = map_score
        epochs_since_improvement = 0
        torch.save(model.state_dict(), "best_model_new.pth")
        print("✅ Model improved — saved as best_model.pth")
    else:
        epochs_since_improvement += 1
        print("⚠️  No improvement this epoch.")

    if epochs_since_improvement >= patience:
        print("\n⏹️ Early stopping triggered — no improvement for", patience, "epochs.")
        break
     # Logging
    print(f"🔍 mAP@0.5 this epoch   : {map_score:.4f}")
    print(f"📈 Best mAP so far     : {best_map:.4f}")
    print(f"📌 Epochs w/o improve  : {epochs_since_improvement}/{patience}")


📘 Epoch 1/100


🔍 mAP@0.5 this epoch   : 0.0111
📈 Best mAP so far     : 0.0000
📌 Epochs w/o improve  : 0/10
✅ Model improved — saved as best_model.pth
🔍 mAP@0.5 this epoch   : 0.0111
📈 Best mAP so far     : 0.0111
📌 Epochs w/o improve  : 0/10

📘 Epoch 2/100


🔍 mAP@0.5 this epoch   : 0.1500
📈 Best mAP so far     : 0.0111
📌 Epochs w/o improve  : 0/10
✅ Model improved — saved as best_model.pth
🔍 mAP@0.5 this epoch   : 0.1500
📈 Best mAP so far     : 0.1500
📌 Epochs w/o improve  : 0/10

📘 Epoch 3/100


🔍 mAP@0.5 this epoch   : 0.2262
📈 Best mAP so far     : 0.1500
📌 Epochs w/o improve  : 0/10
✅ Model improved — saved as best_model.pth
🔍 mAP@0.5 this epoch   : 0.2262
📈 Best mAP so far     : 0.2262
📌 Epochs w/o improve  : 0/10

📘 Epoch 4/100


🔍 mAP@0.5 this epoch   : 0.3074
📈 Best mAP so far     : 0.2262
📌 Epochs w/o improve  : 0/10
✅ Model improved — saved as best_model.pth
🔍 mAP@0.5 this epoch   : 0.3074
📈 Best mAP so far     : 0.3074
📌 Epochs w/o improve  : 0/10

📘 Epoch 5/100


...

🔍 mAP@0.5 this epoch   : 0.8358
📈 Best mAP so far     : 0.8579
📌 Epochs w/o improve  : 5/10
⚠️  No improvement this epoch.
🔍 mAP@0.5 this epoch   : 0.8358
📈 Best mAP so far     : 0.8579
📌 Epochs w/o improve  : 6/10

📘 Epoch 57/100


🔍 mAP@0.5 this epoch   : 0.8307
📈 Best mAP so far     : 0.8579
📌 Epochs w/o improve  : 6/10
⚠️  No improvement this epoch.
🔍 mAP@0.5 this epoch   : 0.8307
📈 Best mAP so far     : 0.8579
📌 Epochs w/o improve  : 7/10

📘 Epoch 58/100


🔍 mAP@0.5 this epoch   : 0.8294
📈 Best mAP so far     : 0.8579
📌 Epochs w/o improve  : 7/10
⚠️  No improvement this epoch.
🔍 mAP@0.5 this epoch   : 0.8294
📈 Best mAP so far     : 0.8579
📌 Epochs w/o improve  : 8/10

📘 Epoch 59/100


🔍 mAP@0.5 this epoch   : 0.8119
📈 Best mAP so far     : 0.8579
📌 Epochs w/o improve  : 8/10
⚠️  No improvement this epoch.
🔍 mAP@0.5 this epoch   : 0.8119
📈 Best mAP so far     : 0.8579
📌 Epochs w/o improve  : 9/10

📘 Epoch 60/100


🔍 mAP@0.5 this epoch   : 0.8334
📈 Best mAP so far     : 0.8579
📌 Epochs w/o improve  : 9/10
⚠️  No improvement this epoch.

⏹️ Early stopping triggered — no improvement for 10 epochs.
